In [66]:
import copy


class WumpusPuzzle:
    def __init__(self, s, a, q, qa, p):
        self.size = s
        self.arrows = a
        self.query = q
        self.queryAnswer = qa
        self.path = p

In [67]:
class Clause:
    def __init__(self, l, c):
        #list of literals
        self.literals = l
        #booleans
        self.isDisjunction = not c
        self.isConjunction = c

    def __str__(self):
        finalStr = ""
        for i in range(len(self.literals)):
            finalStr += str(i) +". " + str(self.literals[i].negation) + " function " + str(self.literals[i].predicate.function) + ", variables: " + str(self.literals[i].predicate.variables) + "\n"
        return finalStr

In [68]:
class Literal:
    def __init__(self, p, n):
        #boolean
        self.negation = n
        #predicate
        self.predicate = p

    def getValue(self):
        if(self.predicate.truthValue == None):
            return None
        if self.negation:
            return self.predicate.truthValue
        else:
            return not self.predicate.truthValue

    def __eq__(self, other):
        if self.predicate.equalsOther(other.predicate) and self.negation == other.negation and variablesEqual(self, other):
            return True
        else:
            return False


In [69]:
def literalsCancel(literal1, literal2):
    if((literal1.predicate.equalsOther(literal2.predicate)) and (literal1.negation != literal2.negation)):
        return True
    else:
        return False

def variablesEqual(literal1, literal2):
    for var in range(len(literal1.predicate.variables)):
            if literal1.predicate.variables[var].name != literal2.predicate.variables[var].name:
                return False
    return True

In [70]:
class Predicate:
    def __init__(self, v, func):
        self.variables = v
        self.function = func
        self.truthValue = func(v)

    def equalsOther(self, otherPredicate):
        if(self.function == otherPredicate.function):
            return True
        return False

    def replaceVariables(self, dict):
        for replacementVariable in dict.keys():
            for predVariable in range(len(self.variables)):
                print()
                if(self.variables[predVariable] == replacementVariable):
                    print(self.variables[predVariable])
                    self.variables[predVariable] = dict[replacementVariable]
                    print(self.variables[predVariable])


In [71]:
def Cat(value):
    if(value[0].name == 'Tuna'):
        return True
    if(value[0].name == None):
        return None
    return False


In [72]:
def Loves(values):
    if(values[1] in values[0].loves):
        return True
    return False


In [73]:
def Animal(values):
    if(values[0].isAnimal):
        return True
    return False

In [74]:
def Kills(values):
    if(values[1] in values[0].kills):
        return True
    return False

In [75]:
class toyVariable:
    def __init__(self, n, a, l, k, v):
        #string
        self.name = n
        #boolean
        self.isAnimal = a
        #lists
        self.loves = l
        self.kills = k
        self.isVariable = v

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

    def __eq__(self,other):
        return self.name == other.name

    def __hash__(self):
        return hash(self.name)




In [76]:
def resolution(KB, query):
    clauses = copy.deepcopy(KB)
    clauses.add(query)
    new = set()
    while(True):
        clausePairs = set()
        #setting up all pairs of clauses
        for clause in clauses:
            for clause2 in clauses:
                if(clause != clause2):
                    clausePairs.add(tuple([clause, clause2]))
        for pair in clausePairs:
            print("clauses before: ", pair[0], pair[1])
            resolvent = resolve(pair[0], pair[1])
            print("clause after: ", resolvent)
            if(len(resolvent.literals) == 0):
                return True
            else:
                new.add(resolvent)
        if(new.issubset(clauses)):
            return False
        clauses = clauses.union(new)



def resolve(c1, c2):
    clause1 = copy.deepcopy(c1)
    clause2 = copy.deepcopy(c2)
    newLiterals = []
    newLiterals += clause1.literals + clause2.literals
    for literal1 in clause1.literals:
        for literal2 in clause2.literals:
            if(literalsCancel(literal1,literal2)):
                theta = {}
                theta = unify(literal1, literal2, theta)
                print("theta:", theta)
                if(theta != "fail"):
                    #replacing with substitutions
                    for literalSub1 in clause1.literals:
                        for literalSub2 in clause2.literals:
                            literalSub1.predicate.replaceVariables(theta)
                            literalSub2.predicate.replaceVariables(theta)
                if(variablesEqual(literal1, literal2)):
                    newLiterals.remove(literal1)
                    newLiterals.remove(literal2)


    newClause = Clause(newLiterals, False)
    return newClause







In [77]:
# Perform substitution to make x and y the same
def unify(x, y, theta):
    if theta == "fail":
        return "fail"

    if (x == y):
        return theta

    elif (isinstance(x,toyVariable) and x.isVariable):
        return unifyVar(x, y, theta)
    elif (isinstance(y,toyVariable) and y.isVariable):
        return unifyVar(y, x, theta)

    elif isinstance(x, Predicate) and isinstance(y, Predicate):
        if x.function != y.function or len(x.variables) != len(y.variables):
            return "fail"
        return unify(x.variables, y.variables, theta)

    elif isinstance(x, Literal) and isinstance(y, Literal):
        return unify(x.predicate, y.predicate, theta)

    elif isinstance(x, Clause) and isinstance(y, Clause):
        if len(x.literals) != len(y.literals):
            return "fail"
        for l1, l2 in zip(x.literals, y.literals):
            theta = unify(l1.predicate, l2.predicate, theta)

        return theta

    elif isinstance(x, list) and isinstance(y, list):
        if len(x) != len(y):
            return "fail"
        for i, j in zip(x, y):
            theta = unify(i, j, theta)
            if theta == "fail":
                return "fail"
        return theta

    else:
        return "fail"




def unifyVar(var, x, theta):
    if var in theta:
        return unify(theta[var],x,theta)
    elif x in theta:
        return unify(var,theta[x],theta)
    #Maybe need to have the occur-check call here and write occur-check function? Says not always done in the textbook
    else:
        new = theta.copy()
        new[var] = x
        return new



def unifyV2(literal1, literal2):
    if(literal1.predicate.variables == literal2.predicate.variables):
        return True
    else:
        newLit1 = copy.deepcopy(literal1)
        newLit2 = copy.deepcopy(literal2)
        for i in range(len(literal1.predicate.variables)):
            if(literal1.predicate.variables[i].isVariable):
                newLit1.predicate.variables[i] = literal2.predicate.variables[i]
            elif(literal1.predicate.variables[i] != literal2.predicate.variables[i]):
                if(literal2.predicate.variables[i].isVariable):
                    newLit2.predicate.variables[i] = literal1.predicate.variables[i]
                else:
                    return False
        return True



In [78]:
PuzzleFile = 'Caves/_wumpus_paths_/easy/path_e3.txt'

def processFile(fileName):
    with open(fileName, 'r', encoding='utf-8-sig') as file:
        content = file.read().strip()
        result = content.split("\n")
        result2 = result[0].split("GRID: ")
        xAxis = int(result2[1].split("x")[0])
        yAxis = int(result2[1].split("x")[1])
        size = [xAxis, yAxis]
        result3 = result[1].split("ARROWS: ")
        arrows = int(result3[1])
        index = 3
        path = result[index]
        pathArr = []
        while(path != ''):
            pathStr = path.strip().split("(")[1].split(")")
            xVal = int(pathStr[0].split(",")[0])
            yVal = int(pathStr[0].split(",")[1])
            breezeVal = pathStr[1].split("B:")[1][0]
            stenchVal = pathStr[1].split("S:")[1][0]
            pathVal = [(xVal, yVal), breezeVal, stenchVal]
            pathArr.append(pathVal)
            index+=1
            path = result[index]
        queryResult = result[index+1].split("QUERY: ")[1].split(',')
        query = [int(queryResult[0][1:]), int(queryResult[1][:-1])]
        resolution = result[index+2].split("RESOLUTION:")[1].strip()
        puzzle = WumpusPuzzle(size, arrows, query, resolution, pathArr)
        print(size)
        print(arrows)
        print(pathArr)
        print(query)
        print(resolution)
        return puzzle

puzzle = processFile(PuzzleFile)

[20, 20]
2
[[(0, 0), 'F', 'F'], [(1, 0), 'F', 'T'], [(2, 0), 'F', 'F'], [(2, 1), 'F', 'T'], [(3, 1), 'F', 'T'], [(4, 1), 'F', 'F'], [(4, 2), 'F', 'T'], [(5, 2), 'F', 'F'], [(5, 3), 'T', 'F'], [(4, 3), 'F', 'F'], [(3, 3), 'T', 'T'], [(2, 3), 'F', 'F'], [(2, 2), 'F', 'T']]
[1, 2]
SAFE


In [79]:


KB = set()

Tuna = toyVariable("Tuna", None, [], [], False)
Jack = toyVariable("Jack", None, [], [], False)
Curiosity = toyVariable("Curiosity", None, [], [], False)
x = toyVariable("x", True, [], [], True)
fx = toyVariable("fx", True, [], [], True)
gx = toyVariable("gx", True, [], [], True)
y = toyVariable("y", True, [], [], True)
z = toyVariable("z", True, [], [], True)



p = Predicate([Tuna], Cat)
l = Literal(p, True)
rule6 = Clause([l], False)

p = Predicate([x], Cat)
l = Literal(p, False)
p = Predicate([x], Animal)
l2 = Literal(p, True)
rule7 = Clause([l,l2], False)

p = Predicate([fx], Animal)
l = Literal(p, True)
p = Predicate([gx, x], Loves)
l2 = Literal(p, True)
rule1 = Clause([l,l2], False)


p = Predicate([x, fx], Loves)
l = Literal(p, False)
p = Predicate([gx, x], Loves)
l2 = Literal(p, True)
rule2 = Clause([l,l2], False)

p = Predicate([y, x], Loves)
l = Literal(p, False)
p = Predicate([z], Animal)
l2 = Literal(p, False)
p = Predicate([x,z], Kills)
l3 = Literal(p, False)
rule3 = Clause([l,l2,l3], False)

p = Predicate([x], Animal)
l = Literal(p, False)
p = Predicate([Jack, x], Loves)
l2 = Literal(p, True)
rule4 = Clause([l,l2], False)

p = Predicate([Jack,Tuna], Kills)
l = Literal(p, True)
p = Predicate([Curiosity,Tuna], Kills)
l2 = Literal(p, True)
rule5 = Clause([l,l2], False)

KB.add(rule1)
KB.add(rule2)
KB.add(rule3)
KB.add(rule4)
KB.add(rule5)
KB.add(rule6)
KB.add(rule7)

p = Predicate([Curiosity, Tuna], Kills)
l = Literal(p, False)
query = Clause([l], False)

#Testing literal cancel checking function
p = Predicate([Jack,Tuna], Kills)
l = Literal(p, True)
p = Predicate([Jack,Tuna], Kills)
l2 = Literal(p, False)
print(literalsCancel(l,l2))

#Testing unification
# Variables
i = toyVariable("i", True, [], [], True)
j = toyVariable("j", True, [], [], True)
Cow = toyVariable("Cow", None, [], [], False)

p1 = Predicate([j, Cow], Cat)
p2 = Predicate([j, i], Cat)
p3 = Predicate([i], Cat)

# Literals
l1 = Literal(p1, False)
l2 = Literal(p2, False)
l3 = Literal(p3, True)

# Clauses
c1 = Clause([l1], False)
c2 = Clause([l2], False)
c3 = Clause([l2, l3], False)

# --- Test ---
theta = {}
print("Unify Literals:", unify(l1, l2, theta))
print("Unify Predicates:", unify(p1, p2, theta))
print("Unify Clauses:")
for l in rule7.literals:
    for l2 in rule6.literals:
        print(unify(l, l2, theta))


#
# newClauses = resolve(rule5, query)
print(rule6)
print(rule7)
newClauses = resolve(rule6, rule7)
print(newClauses)
print(rule3)
newerClauses = resolve(newClauses, rule3)
print(newerClauses)
newerClauses2 = resolve(rule5, query)
print(newerClauses2)
tester = resolution(KB, query)
final = resolve(newerClauses, newerClauses2)
print(final)

True
Unify Literals: {i: Cow}
Unify Predicates: {i: Cow}
Unify Clauses:
{x: Tuna}
fail
0. True function <function Cat at 0x00000276530A6700>, variables: [Tuna]

0. False function <function Cat at 0x00000276530A6700>, variables: [x]
1. True function <function Animal at 0x0000027651E46DE0>, variables: [x]

theta: {x: Tuna}


x
Tuna


x
Tuna
0. True function <function Animal at 0x0000027651E46DE0>, variables: [Tuna]

0. False function <function Loves at 0x00000276530A4860>, variables: [y, x]
1. False function <function Animal at 0x0000027651E46DE0>, variables: [z]
2. False function <function Kills at 0x000002765303C900>, variables: [x, z]

theta: {z: Tuna}





z
Tuna



z
Tuna
0. False function <function Loves at 0x00000276530A4860>, variables: [y, x]
1. False function <function Kills at 0x000002765303C900>, variables: [x, Tuna]

theta: fail
theta: {}
0. True function <function Kills at 0x000002765303C900>, variables: [Jack, Tuna]

clauses before:  0. True function <function Kills at 0x0

ValueError: list.remove(x): x not in list